In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import numpy as np
import re
import urllib.request
from bs4 import BeautifulSoup

In [2]:
cid = 'client-id'
secret = 'secret-id'
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Reading data from the following playlists: <br>
https://open.spotify.com/playlist/1Csg2osWLx5EOa3bTVvKxg <br>
https://open.spotify.com/playlist/3pL68OCEZXmx6IBKggVdgR <br>
https://open.spotify.com/playlist/2bFHkH8xHkHGPpfnhQQFn5 <br>
https://open.spotify.com/playlist/6nXSobvQDOEXkjlxE19o0b <br>

In [3]:
playlists_ids = ['1Csg2osWLx5EOa3bTVvKxg',
                '3pL68OCEZXmx6IBKggVdgR',
                '2bFHkH8xHkHGPpfnhQQFn5',
                '6nXSobvQDOEXkjlxE19o0b']
playlists = []

for i in range(1, len(playlists_ids)):
    playlists.append(sp.playlist_tracks(playlist_id=playlists_ids[i], limit=100))

In [4]:
artist_name = []
track_name = []
track_duration = []
track_album = []
album_release = []
album_release_precision = []
track_id = []
track_popularity = []

for i in range(0, len(playlists)):
    for item in playlists[i]['items']:
        track_name.append(item['track']['name'])
        track_duration.append(item['track']['duration_ms'])
        artist_name.append(item['track']['artists'][0]['name'])
        track_album.append(item['track']['album']['name'])
        album_release.append(item['track']['album']['release_date'])
        album_release_precision.append(item['track']['album']['release_date_precision'])
        track_id.append(item['track']['id'])
        track_popularity.append(item['track']['popularity'])

In [5]:
peru_df = pd.DataFrame({'artist_name' : artist_name, 
                        'track_name' : track_name, 
                        'track_duration' : track_duration, 
                        'track_album' : track_album,
                        'track_id' : track_id,
                        'album_release' : album_release,
                        'album_release_precision' : album_release_precision,
                        'track_popularity' : track_popularity})
peru_df.head()

,artist_name,track_name,track_duration,track_album,track_id,album_release,album_release_precision,track_popularity
0,Dalevuelta,Quiero Ser un Ramones,235466,Somos Lo Que Fuimos,7IR1I8gbww7Bfe8tqY4jHi,2015-08-26,day,9
1,Dalevuelta,Un Simple Periodo,193021,Somos Lo Que Fuimos,0EKSdRveLQHhtntvJWFHWI,2015-08-26,day,6
2,Dalevuelta,Y Tu No Estás,151791,Somos Lo Que Fuimos,0pVED3J7TWoKzLBSX7OzeK,2015-08-26,day,8
3,Dalevuelta,A un Amigo,210058,Somos Lo Que Fuimos,6mVInhdzGpqElPqD2LnJ6D,2015-08-26,day,7
4,Dalevuelta,Inocencia,143072,Antología (1999 - 2008),7nTyWxrQYwANc2VtK9PRMw,2014-06-30,day,9


In [6]:
peru_df.shape

(215, 8)

In [17]:
## Function to return lyrics from AZ lyrics
## src: https://www.quora.com/Whats-a-good-API-to-use-to-get-song-lyrics

def get_lyrics(artist,song_title):
    artist = artist.lower()
    song_title = song_title.lower()
    # remove all except alphanumeric characters from artist and song_title
    artist = re.sub('[^A-Za-z0-9]+', "", artist)
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who
        artist = artist[3:]
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"
    
    try:
        content = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(content, 'html.parser')
        lyrics = str(soup)
        # lyrics lies between up_partition and down_partition
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->'
        down_partition = '<!-- MxM banner -->'
        lyrics = lyrics.split(up_partition)[1]
        lyrics = lyrics.split(down_partition)[0]
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').strip()
        return lyrics
    except Exception as e:
        return "Exception occurred \n" +str(e)

In [18]:
get_lyrics(peru_df['artist_name'][0], peru_df['track_name'][0])

'Exception occurred \nHTTP Error 404: Not Found'